In [1]:
'''
The thermal face recognition process is undertaken using convolutional neural networks (CNN's). 
More precisely, the system comprises the first 10 layers from the VGG16 architecture, followed by 
a max-pooling layer, a batch-normalization layer and a classifier (densely connected layer).'''

In [2]:
#The VGG16 architecture is imported.
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(72, 96, 3))

Using TensorFlow backend.
/home/rauly/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/rauly/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 72, 96, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 72, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 72, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 36, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 36, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 36, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 18, 24, 128)       0         
__________

In [ ]:
#From the VGG16 model, only the first 10 layers are used.

In [4]:
for i in range (1,10):
    conv_base.layers.pop()

In [5]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 72, 96, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 72, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 72, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 36, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 36, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 36, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 18, 24, 128)       0         
__________

In [6]:
inp = conv_base.input
out =conv_base.layers[-1].output

In [8]:
from keras.models import Model
model2 = Model(inp, out)

In [ ]:
#From the first 10 layers, only the last three are set up to allow the training.

In [9]:
cont = 0
for layer in model2.layers:
    cont = cont + 1
    if (cont >= 8):
        layer.trainable = True
    else:
        layer.trainable = False

In [10]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 72, 96, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 72, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 72, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 36, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 36, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 36, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 18, 24, 128)       0         
__________

In [11]:
#Copying images to training, validation, and test directories
import os, shutil
import numpy as np
#Path where my dataset is stored
base_dir = '/home/rauly/Documents/Terravic_Thermal_Full/Full_Sections'

# Directories for the training, validation and test splits
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

In [12]:
#Here is the proposed model. After the modified VGG16 architecture, a max-pooling layer is append, followed by a 
#batch-normalization layer, and before the addition of the softmax classifier with 18 categories, the model
#is properly flatten.
from keras import layers
from keras import models
from keras import regularizers
from keras.layers.normalization import BatchNormalization

model = models.Sequential()
model.add(model2)
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.Flatten())
model.add(layers.Dense(18, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 18, 24, 256)       1735488   
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 12, 256)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 12, 256)        1024      
_________________________________________________________________
flatten_1 (Flatten)          (None, 27648)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 18)                497682    
Total params: 2,234,194
Trainable params: 1,973,522
Non-trainable params: 260,672
_________________________________________________________________


In [13]:
#The module metrics is imported in order to show the rank 1-3 recognition rates
from keras import metrics
import functools
from functools import partial
top2_acc = functools.partial(metrics.top_k_categorical_accuracy, k=2)
top3_acc = functools.partial(metrics.top_k_categorical_accuracy, k=3)
top2_acc.__name__ = 'top2_acc'
top3_acc.__name__ = 'top3_acc'

In [14]:
#Compilation stage
from keras import optimizers

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4), #Decrease learning rate
              metrics=['accuracy',top2_acc, top3_acc])

In [15]:
#Using ImageDataGenerator to read images from directories
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir, # Target directory
        target_size=(72, 96), # All images are resized from 240x320 to 72x96
        batch_size= 10, 
        color_mode='rgb',
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(72, 96),
        batch_size=144,
        color_mode='rgb',
        class_mode='categorical')

Found 1710 images belonging to 18 classes.
Found 576 images belonging to 18 classes.


In [16]:
#Training and validation stages
history = model.fit_generator(
      train_generator,
      steps_per_epoch=171, #70
      epochs= 10,
      validation_data=validation_generator,
      validation_steps=4,
      workers=0,
      max_queue_size=0)

Epoch 1/10
171/171 [==============================] - 107s 628ms/step - loss: 0.3841 - acc: 0.9082 - top2_acc: 0.9409 - top3_acc: 0.9567 - val_loss: 0.0234 - val_acc: 0.9931 - val_top2_acc: 0.9983 - val_top3_acc: 1.0000
Epoch 2/10
171/171 [==============================] - 110s 645ms/step - loss: 0.0065 - acc: 0.9977 - top2_acc: 1.0000 - top3_acc: 1.0000 - val_loss: 0.0044 - val_acc: 0.9983 - val_top2_acc: 1.0000 - val_top3_acc: 1.0000
Epoch 3/10
171/171 [==============================] - 113s 659ms/step - loss: 5.9952e-04 - acc: 1.0000 - top2_acc: 1.0000 - top3_acc: 1.0000 - val_loss: 0.0052 - val_acc: 0.9983 - val_top2_acc: 1.0000 - val_top3_acc: 1.0000
Epoch 4/10
171/171 [==============================] - 117s 682ms/step - loss: 7.1040e-05 - acc: 1.0000 - top2_acc: 1.0000 - top3_acc: 1.0000 - val_loss: 1.9285e-04 - val_acc: 1.0000 - val_top2_acc: 1.0000 - val_top3_acc: 1.0000
Epoch 5/10
171/171 [==============================] - 111s 647ms/step - loss: 3.5481e-06 - acc: 1.0000 - top

In [22]:
#The proposed model is defined again
model = models.Sequential()
model.add(model2)
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.Flatten())
model.add(layers.Dense(18, activation='softmax'))

In [23]:
#Compilation stage
from keras import optimizers

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4), #Decrease learning rate
              metrics=['accuracy',top2_acc, top3_acc])

In [24]:
#Retraining from scratch
history = model.fit_generator(
      train_generator,
      steps_per_epoch=292,
      epochs=4,
      workers=0,
      max_queue_size=0)

Epoch 1/4
292/292 [==============================] - 171s 584ms/step - loss: 0.1663 - acc: 0.9606 - top2_acc: 0.9740 - top3_acc: 0.9818
Epoch 2/4
292/292 [==============================] - 165s 565ms/step - loss: 2.6602e-04 - acc: 1.0000 - top2_acc: 1.0000 - top3_acc: 1.0000
Epoch 3/4
292/292 [==============================] - 161s 553ms/step - loss: 2.6214e-07 - acc: 1.0000 - top2_acc: 1.0000 - top3_acc: 1.0000
Epoch 4/4
292/292 [==============================] - 160s 549ms/step - loss: 1.2094e-07 - acc: 1.0000 - top2_acc: 1.0000 - top3_acc: 1.0000


In [25]:
#Test stage
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(72, 96),
        batch_size=180,
        color_mode='rgb',
        class_mode='categorical')

test_loss, test_acc, test_top2_acc, test_top3_acc= model.evaluate_generator(test_generator, steps=10, workers=0, max_queue_size=0)
print('Recognition rate top1: ', test_acc)
print('Recognition rate top2: ', test_top2_acc)
print('Recognition rate top3: ', test_top3_acc)

Found 1800 images belonging to 18 classes.
Recognition rate top1:  1.0
Recognition rate top2:  1.0
Recognition rate top3:  1.0
